In [1165]:
import pandas as pd
import numpy as np
import requests as rq
from io import BytesIO
import json

# 주식정보 변환

In [1166]:
etf_info = pd.read_csv('날짜별etf정보.csv')
stock_info = pd.read_csv('날짜별주식정보.csv')
etf_price = pd.read_csv('etf가격정보.csv')

In [1167]:
stock_info[['종목코드', '날짜']] = stock_info['종목코드 + 날짜'].str.split('/',expand = True)

In [1168]:
stock_info = stock_info.drop(columns = ['종목코드 + 날짜', '종목명', '대비','등락률'])

In [1169]:
stock_info['종목코드'] = stock_info['종목코드'].astype(str).str.zfill(6)

In [1170]:
filter = stock_info['날짜'] > '20181201'
stocks = stock_info[filter]

In [1171]:
stocks

,시장구분,업종명,종가,시가총액,종목코드,날짜
0,KOSPI,서비스업,4865,220154672535,095570,20240329
1,KOSPI,기타금융,15100,200038171100,006840,20240329
2,KOSPI,기타금융,3740,357980798340,027410,20240329
3,KOSPI,유통업,117700,2034315736200,282330,20240329
4,KOSPI,기타금융,7770,2502627163260,138930,20240329
...,...,...,...,...,...,...
22906,KOSDAQ,유통,3695,55425000000,024060,20190329
22907,KOSDAQ,기계·장비,6850,84410467600,010240,20190329
22908,KOSDAQ,음식료·담배,7280,53444598480,189980,20190329
22909,KOSDAQ,기타서비스,4755,66201368625,037440,20190329


# 네이버에서 가치지표 채우기

In [1172]:
stocks.loc[:, '주식 종류'] = np.where(stocks['종목코드'].str[-1] != '0', '우선주', '보통주')

C:\Users\user\AppData\Local\Temp\ipykernel_27820\61550883.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks.loc[:, '주식 종류'] = np.where(stocks['종목코드'].str[-1] != '0', '우선주', '보통주')


In [1173]:
stocks.loc[:,'PER'] = None
stocks.loc[:,'PBR'] = None
stocks.loc[:,'PCR'] = None
stocks.loc[:,'PSR'] = None
stocks.loc[:,'DPS'] = None

C:\Users\user\AppData\Local\Temp\ipykernel_27820\2158394025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks.loc[:,'PER'] = None
C:\Users\user\AppData\Local\Temp\ipykernel_27820\2158394025.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks.loc[:,'PBR'] = None
C:\Users\user\AppData\Local\Temp\ipykernel_27820\2158394025.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [1214]:
def get_value_index(ticker):
    
    session = requests.Session()
    
    # 쿠키 정보를 세션에 추가합니다
    # https://finance.naver.com/item/coinfo.naver?code=005930사이트는 여기에 투자지표사이트로 들어가야 함
    # encparam으로 값 update필요
    
    session.cookies.set('setC1010001', '%5B%7B%22conGubun%22%3A%22MAIN%22%2C%22cTB23%22%3A%22cns_td1%22%2C%22bandChartGubun%22%3A%22MAIN%22%2C%22finGubun%22%3A%22MAIN%22%2C%22cTB00%22%3A%22cns_td20%22%7D%5D')
    session.cookies.set('setC1030001', '%5B%7B%22cTB301%22%3A%22rpt_td2%22%2C%22finGubun%22%3A%22MAIN%22%2C%22frqTyp%22%3A%220%22%2C%22moreY%22%3A1%2C%22moreQ%22%3A1%2C%22moreQQ%22%3A1%7D%5D')
    session.cookies.set('setC1040001', '%5B%7B%22cTB401%22%3A%22val_td1%22%2C%22finGubun%22%3A%22MAIN%22%2C%22finGubun2%22%3A%22MAIN%22%2C%22frqTyp%22%3A%220%22%2C%22frqTyp_2%22%3A%220%22%2C%22moreY%22%3A1%2C%22moreQ%22%3A1%2C%22moreQQ%22%3A1%2C%22moreY2%22%3A1%2C%22moreQ2%22%3A1%2C%22moreQQ2%22%3A1%7D%5D')
    
    
    url = f'https://navercomp.wisereport.co.kr/v2/company/cF4002.aspx?cmp_cd={ticker}&frq=0&rpt=5&finGubun=MAIN&frqTyp=0&cn=&encparam=SnI1SFc0RENEYW5xdUpuM0RJYVpadz09'
    
    # 웹 페이지 요청 및 HTML 파싱
    # session.headers.update({'Referer': 'https://navercomp.wisereport.co.kr/v2/company/c1040001.aspx?cmp_cd=000660&cn='})
    session.headers.update({'Referer': 'https://navercomp.wisereport.co.kr/v2/company/c1040001.aspx'})
    
    # 웹 페이지 요청 및 HTML 파싱
    response = session.get(url)
    print(response.text)
    
    
    data = json.loads(response.text)['DATA']
    df = pd.json_normalize(data)
    
    result = {
        'PER' : df.iloc[16,7:12].values,
        'PBR' : df.iloc[19,7:12].values,
        'PCR' : df.iloc[22,7:12].values,
        'PSR' : df.iloc[25,7:12].values,
        'DPS' : df.iloc[31,7:12].values
    }
    dates = ['20200331', '20210331', '20220331', '20230331', '20240329']
    
    return pd.DataFrame(result, index = dates)

In [1215]:
tickers = stocks['종목코드'].unique()
dates = stocks['날짜'].unique()
value_index = ['PER', 'PBR', 'PCR', 'PSR', 'DPS']

for ticker in tickers:
    values = get_value_index(ticker)

    for date in dates:
        for index in value_index:
            mask = (stocks['종목코드'] == ticker) & (stocks['날짜'] == date)
            stocks.loc[mask, index] = values.loc[date, index]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

#### 값이 안채워진 것들 중 우선주값 채우기
#### dps가 null인 경우가 많음 -> dps컬럼 지우기
#### 상장이 최근에 된 기업들이나 가치 지표가 없는 기업들이 있음 -> 어떻게 처리할지 고려해보기

In [599]:
stocks[stocks.isnull().any(axis = 1)]

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR,DPS
6,KOSPI,섬유의복,41720074500,001465,20240329,우선주,None,None,None,None,None
9,KOSPI,기타금융,384612592000,00104K,20240329,우선주,None,None,None,None,None
11,KOSPI,음식료품,97013087100,011150,20240329,보통주,23.94928,1.36097,19.27478,0.71605,None
12,KOSPI,음식료품,4120000000,011155,20240329,우선주,None,None,None,None,None
13,KOSPI,기타금융,143524160500,001045,20240329,우선주,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
21543,KOSDAQ,음식료·담배,80762168880,008800,20200331,보통주,None,None,None,None,None
21551,KOSDAQ,금속,23089235302,053660,20200331,보통주,None,None,None,None,None
21555,KOSDAQ,기계·장비,44712668000,039610,20200331,보통주,20.743,0.84176,11.57711,0.89344,None
21559,KOSDAQ,운송장비·부품,73560710880,134780,20200331,보통주,None,None,None,None,None


In [600]:
stocks[stocks.isnull().any(axis = 1)& (stocks.isnull().sum(axis=1) > 1) & (stocks['주식 종류'] == '보통주')]

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR,DPS
159,KOSPI,건설업,328192582800,475150,20240329,보통주,NaN,NaN,NaN,NaN,NaN
380,KOSPI,기타금융,5578001344000,088980,20240329,보통주,NaN,None,NaN,NaN,None
381,KOSPI,기타금융,404757683160,094800,20240329,보통주,NaN,None,NaN,NaN,None
424,KOSPI,서비스업,19208323080,030790,20240329,보통주,NaN,None,NaN,NaN,None
588,KOSPI,화학,1896094500000,278470,20240329,보통주,NaN,0.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...
21538,KOSDAQ,운송장비·부품,81066045500,102210,20200331,보통주,None,None,None,None,None
21539,KOSDAQ,유통,226999967740,220630,20200331,보통주,None,None,None,None,None
21543,KOSDAQ,음식료·담배,80762168880,008800,20200331,보통주,None,None,None,None,None
21551,KOSDAQ,금속,23089235302,053660,20200331,보통주,None,None,None,None,None


In [601]:
stocks[stocks.isnull().any(axis = 1) & (stocks['주식 종류'] == '우선주')]

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR,DPS
6,KOSPI,섬유의복,41720074500,001465,20240329,우선주,None,None,None,None,None
9,KOSPI,기타금융,384612592000,00104K,20240329,우선주,None,None,None,None,None
12,KOSPI,음식료품,4120000000,011155,20240329,우선주,None,None,None,None,None
13,KOSPI,기타금융,143524160500,001045,20240329,우선주,None,None,None,None,None
15,KOSPI,음식료품,183185754000,097955,20240329,우선주,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
19014,KOSDAQ,의료·정밀기기,7762017000,08537M,20210331,우선주,None,None,None,None,None
19240,KOSDAQ,유통,3592853550,032685,20210331,우선주,None,None,None,None,None
20399,KOSDAQ,금속,5281870170,021045,20200331,우선주,None,None,None,None,None
20499,KOSDAQ,의료·정밀기기,5725096300,08537M,20200331,우선주,None,None,None,None,None


In [602]:
stocks[(stocks['DPS'].isnull()) & (stocks.isnull().sum(axis=1) == 1) & (stocks['주식 종류'] == '보통주')]

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR,DPS
11,KOSPI,음식료품,97013087100,011150,20240329,보통주,23.94928,1.36097,19.27478,0.71605,NaN
18,KOSPI,서비스업,311417248284,012030,20240329,보통주,17.19465,0.83827,6.64761,0.76024,NaN
33,KOSPI,화학,865143230400,017860,20240329,보통주,36.33066,6.1075,40.61663,1.35838,NaN
40,KOSPI,건설업,1279443275500,006360,20240329,보통주,-2.66718,0.29553,2.73614,0.09567,NaN
48,KOSPI,기타금융,8379536934400,009540,20240329,보통주,38.5929,0.86325,4.11054,0.40178,NaN
...,...,...,...,...,...,...,...,...,...,...,...
21500,KOSDAQ,유통,16918817670,039740,20200331,보통주,13.63357,0.60595,276.57312,0.15119,NaN
21502,KOSDAQ,통신서비스,270722079330,025770,20200331,보통주,10.84079,1.27226,4.72818,0.53356,NaN
21533,KOSDAQ,소프트웨어,55305478760,054920,20200331,보통주,1.46111,0.63118,27.63365,3.10945,NaN
21555,KOSDAQ,기계·장비,44712668000,039610,20200331,보통주,20.743,0.84176,11.57711,0.89344,NaN


# dps열 삭제 및 우선주 값 채우는 전처리

In [603]:
stocks = stocks.drop(columns = ['DPS'])

In [604]:
stocks[stocks['주식 종류'] == '우선주'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 604 entries, 6 to 20712
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시장구분    604 non-null    object
 1   업종명     604 non-null    object
 2   시가총액    604 non-null    int64 
 3   종목코드    604 non-null    object
 4   날짜      604 non-null    object
 5   주식 종류   604 non-null    object
 6   PER     0 non-null      object
 7   PBR     0 non-null      object
 8   PCR     0 non-null      object
 9   PSR     0 non-null      object
dtypes: int64(1), object(9)
memory usage: 51.9+ KB


In [605]:
def get_common_stock_value(row):
    common_stock_ticker = row['종목코드'][:5]+'0'
    date = row['날짜']
    values = stocks[(stocks['종목코드'] == common_stock_ticker) & (stocks['날짜'] == date)]

    row[6:] = values.iloc[0,6:]

    return row


In [606]:
stocks.loc[stocks['주식 종류'] == '우선주'] = stocks.loc[stocks['주식 종류'] == '우선주'].apply(get_common_stock_value, axis = 1)

In [607]:
stocks

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR
0,KOSPI,서비스업,220154672535,095570,20240329,보통주,13.95137,0.53399,-12.708,0.23009
1,KOSPI,기타금융,200038171100,006840,20240329,보통주,6.57706,0.38567,0.42958,0.05057
2,KOSPI,기타금융,357980798340,027410,20240329,보통주,4.68667,0.22041,5.18776,0.82289
3,KOSPI,유통업,2034315736200,282330,20240329,보통주,11.58762,2.11104,3.5071,0.27693
4,KOSPI,기타금융,2502627163260,138930,20240329,보통주,3.62053,0.22491,-399.2222,0.23968
...,...,...,...,...,...,...,...,...,...,...
21572,KOSDAQ,유통,79200000000,024060,20200331,보통주,9.69508,1.01218,110.00543,0.53362
21573,KOSDAQ,기계·장비,34688389240,010240,20200331,보통주,7.78644,0.80795,5.05788,0.5219
21574,KOSDAQ,음식료·담배,39092374575,189980,20200331,보통주,12.31997,0.97553,6.63823,1.21519
21575,KOSDAQ,기타서비스,49285561500,037440,20200331,보통주,16.20406,1.01802,2.95334,0.3341


In [608]:
stocks[stocks.isnull().any(axis=1)]

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR
159,KOSPI,건설업,328192582800,475150,20240329,보통주,NaN,NaN,NaN,NaN
380,KOSPI,기타금융,5578001344000,088980,20240329,보통주,NaN,None,NaN,NaN
381,KOSPI,기타금융,404757683160,094800,20240329,보통주,NaN,None,NaN,NaN
424,KOSPI,서비스업,19208323080,030790,20240329,보통주,NaN,None,NaN,NaN
588,KOSPI,화학,1896094500000,278470,20240329,보통주,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
21538,KOSDAQ,운송장비·부품,81066045500,102210,20200331,보통주,None,None,None,None
21539,KOSDAQ,유통,226999967740,220630,20200331,보통주,None,None,None,None
21543,KOSDAQ,음식료·담배,80762168880,008800,20200331,보통주,None,None,None,None
21551,KOSDAQ,금속,23089235302,053660,20200331,보통주,None,None,None,None


##### 45014K는 코오롱모빌리티 , 45226K 한화갤러리 아우, 작년에 상장됨
##### 38380K는 psr값이 2022/3만 비워져 있음
##### 37550K는 2021년 값이 없음
##### 00806K는 2020년 값이 없음
##### 08537m은 없애기

In [609]:
stocks = stocks[stocks['종목코드'] != '08537M']

In [610]:
stocks

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR
0,KOSPI,서비스업,220154672535,095570,20240329,보통주,13.95137,0.53399,-12.708,0.23009
1,KOSPI,기타금융,200038171100,006840,20240329,보통주,6.57706,0.38567,0.42958,0.05057
2,KOSPI,기타금융,357980798340,027410,20240329,보통주,4.68667,0.22041,5.18776,0.82289
3,KOSPI,유통업,2034315736200,282330,20240329,보통주,11.58762,2.11104,3.5071,0.27693
4,KOSPI,기타금융,2502627163260,138930,20240329,보통주,3.62053,0.22491,-399.2222,0.23968
...,...,...,...,...,...,...,...,...,...,...
21572,KOSDAQ,유통,79200000000,024060,20200331,보통주,9.69508,1.01218,110.00543,0.53362
21573,KOSDAQ,기계·장비,34688389240,010240,20200331,보통주,7.78644,0.80795,5.05788,0.5219
21574,KOSDAQ,음식료·담배,39092374575,189980,20200331,보통주,12.31997,0.97553,6.63823,1.21519
21575,KOSDAQ,기타서비스,49285561500,037440,20200331,보통주,16.20406,1.01802,2.95334,0.3341


In [615]:
stocks[stocks.isnull().any(axis=1)]

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR
159,KOSPI,건설업,328192582800,475150,20240329,보통주,NaN,NaN,NaN,NaN
380,KOSPI,기타금융,5578001344000,088980,20240329,보통주,NaN,None,NaN,NaN
381,KOSPI,기타금융,404757683160,094800,20240329,보통주,NaN,None,NaN,NaN
424,KOSPI,서비스업,19208323080,030790,20240329,보통주,NaN,None,NaN,NaN
588,KOSPI,화학,1896094500000,278470,20240329,보통주,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
21538,KOSDAQ,운송장비·부품,81066045500,102210,20200331,보통주,None,None,None,None
21539,KOSDAQ,유통,226999967740,220630,20200331,보통주,None,None,None,None
21543,KOSDAQ,음식료·담배,80762168880,008800,20200331,보통주,None,None,None,None
21551,KOSDAQ,금속,23089235302,053660,20200331,보통주,None,None,None,None


In [617]:
stocks.to_csv('stocks_with_value_index.csv',index = False)

In [645]:
stocks

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR
0,KOSPI,서비스업,220154672535,095570,20240329,보통주,13.95137,0.53399,-12.708,0.23009
1,KOSPI,기타금융,200038171100,006840,20240329,보통주,6.57706,0.38567,0.42958,0.05057
2,KOSPI,기타금융,357980798340,027410,20240329,보통주,4.68667,0.22041,5.18776,0.82289
3,KOSPI,유통업,2034315736200,282330,20240329,보통주,11.58762,2.11104,3.5071,0.27693
4,KOSPI,기타금융,2502627163260,138930,20240329,보통주,3.62053,0.22491,-399.2222,0.23968
...,...,...,...,...,...,...,...,...,...,...
21572,KOSDAQ,유통,79200000000,024060,20200331,보통주,9.69508,1.01218,110.00543,0.53362
21573,KOSDAQ,기계·장비,34688389240,010240,20200331,보통주,7.78644,0.80795,5.05788,0.5219
21574,KOSDAQ,음식료·담배,39092374575,189980,20200331,보통주,12.31997,0.97553,6.63823,1.21519
21575,KOSDAQ,기타서비스,49285561500,037440,20200331,보통주,16.20406,1.01802,2.95334,0.3341


#### 748개의 row가 null값이 있음

In [749]:
len(stocks[stocks.isnull().any(axis=1)]['종목코드'].unique())

418

# fnguide 가치지표에서 지표 가져오기

In [855]:
url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A005930'
response = requests.get(url)

In [856]:
temp = pd.read_html(response.text)[3]

C:\Users\user\AppData\Local\Temp\ipykernel_27820\3825953896.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp = pd.read_html(response.text)[3]


In [857]:
temp = temp.iloc[11:15,1:5]
drop_col = []
for c in temp.columns:
    
    if (c < '2019/12') or (c[5:] != '12'):
        drop_col.append(c)
        print(c)

temp = temp.drop(columns = drop_col)
# print(temp.empty)

# for c in temp.columns:
#     per = temp.loc[11,c]
#     pcr = temp.loc[12,c]
#     psr = temp.loc[13,c]
#     pbr = temp.loc[14,c]
    
#     print(per,pcr,psr,pbr)
temp['2020/12'][11]


'21.09'

In [895]:
date_dic = {
    '20200331': '2019/12',
    '20210331': '2020/12',
    '20220331': '2021/12',
    '20230331': '2022/12',
    '20240329': '2023/12'
}

In [896]:
def get_value_index_fnguide(row):

    if not pd.isna(row['PER']):
        return row
    
    ticker = row['종목코드']
    date = row['날짜']

    try:
        url = f'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A{ticker}'
        response = requests.get(url)
        finan_statement = pd.read_html(response.text)[3]
    
        
        finan_date = date_dic[date]
        per = finan_statement[finan_date][11]
        pcr = finan_statement[finan_date][12]
        psr = finan_statement[finan_date][13]
        pbr = finan_statement[finan_date][14]
        row['PER'] = per
        row['PCR'] = pcr
        row['PSR'] = psr
        row['PBR'] = pbr
    except Exception as e:
        return row
    
    return row
    

In [897]:
temp = stocks.apply(get_value_index_fnguide,axis=1)

C:\Users\user\AppData\Local\Temp\ipykernel_27820\1822264404.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  finan_statement = pd.read_html(response.text)[3]
C:\Users\user\AppData\Local\Temp\ipykernel_27820\1822264404.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  finan_statement = pd.read_html(response.text)[3]
C:\Users\user\AppData\Local\Temp\ipykernel_27820\1822264404.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  finan_statement = pd.read_html(response.text)[3]
C:\Users\user\AppData\Local\Temp\ipykernel_27820\1822264404.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and wil

In [908]:
temp[(0 < temp.isna().sum(axis=1)) & (temp.isna().sum(axis=1) < 4)]

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR
883,KOSPI,서비스업,373474000000,451800,20240329,보통주,NaN,1.04156,48.21701,NaN
1365,KOSPI,서비스업,19208323080,030790,20230331,보통주,NaN,3.24,NaN,0.63
1495,KOSPI,서비스업,400989543110,015540,20230331,보통주,NaN,NaN,NaN,21.60
1739,KOSPI,서비스업,681490397700,950210,20230331,보통주,-3.2542,1.60047,-14.98894,NaN
1999,KOSPI,기타금융,734583044700,383800,20220331,보통주,-18.07566,0.88243,-22.7595,NaN
...,...,...,...,...,...,...,...,...,...,...
21535,KOSDAQ,정보기기,26399800000,066110,20200331,보통주,NaN,1.37,NaN,7.11
21538,KOSDAQ,운송장비·부품,81066045500,102210,20200331,보통주,NaN,0.70,NaN,2.15
21543,KOSDAQ,음식료·담배,80762168880,008800,20200331,보통주,NaN,3.22,NaN,12.64
21551,KOSDAQ,금속,23089235302,053660,20200331,보통주,NaN,NaN,NaN,0.40


In [924]:
temp[temp.isna().any(axis=1)]

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR
159,KOSPI,건설업,328192582800,475150,20240329,보통주,NaN,NaN,NaN,NaN
380,KOSPI,기타금융,5578001344000,088980,20240329,보통주,NaN,NaN,NaN,NaN
381,KOSPI,기타금융,404757683160,094800,20240329,보통주,NaN,NaN,NaN,NaN
424,KOSPI,서비스업,19208323080,030790,20240329,보통주,NaN,NaN,NaN,NaN
588,KOSPI,화학,1896094500000,278470,20240329,보통주,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
21537,KOSDAQ,금융,7059900000,279410,20200331,보통주,None,None,None,None
21538,KOSDAQ,운송장비·부품,81066045500,102210,20200331,보통주,None,0.70,None,2.15
21543,KOSDAQ,음식료·담배,80762168880,008800,20200331,보통주,None,3.22,None,12.64
21551,KOSDAQ,금속,23089235302,053660,20200331,보통주,None,NaN,None,0.40


In [899]:
stocks[stocks.isna().any(axis=1)]

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR
159,KOSPI,건설업,328192582800,475150,20240329,보통주,NaN,NaN,NaN,NaN
380,KOSPI,기타금융,5578001344000,088980,20240329,보통주,NaN,None,NaN,NaN
381,KOSPI,기타금융,404757683160,094800,20240329,보통주,NaN,None,NaN,NaN
424,KOSPI,서비스업,19208323080,030790,20240329,보통주,NaN,None,NaN,NaN
588,KOSPI,화학,1896094500000,278470,20240329,보통주,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
21538,KOSDAQ,운송장비·부품,81066045500,102210,20200331,보통주,None,None,None,None
21539,KOSDAQ,유통,226999967740,220630,20200331,보통주,None,None,None,None
21543,KOSDAQ,음식료·담배,80762168880,008800,20200331,보통주,None,None,None,None
21551,KOSDAQ,금속,23089235302,053660,20200331,보통주,None,None,None,None


In [931]:
stocks = temp

In [1161]:
stocks

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR,종가
0,KOSPI,서비스업,220154672535,095570,20240329,보통주,13.95137,0.53399,-12.708,0.23009,4865
1,KOSPI,기타금융,200038171100,006840,20240329,보통주,6.57706,0.38567,0.42958,0.05057,15100
2,KOSPI,기타금융,357980798340,027410,20240329,보통주,4.68667,0.22041,5.18776,0.82289,3740
3,KOSPI,유통업,2034315736200,282330,20240329,보통주,11.58762,2.11104,3.5071,0.27693,117700
4,KOSPI,기타금융,2502627163260,138930,20240329,보통주,3.62053,0.22491,-399.2222,0.23968,7770
...,...,...,...,...,...,...,...,...,...,...,...
14719,KOSDAQ,유통,55425000000,024060,20190329,NaN,NaN,NaN,NaN,NaN,3695
14720,KOSDAQ,기계·장비,84410467600,010240,20190329,NaN,NaN,NaN,NaN,NaN,6850
14721,KOSDAQ,음식료·담배,53444598480,189980,20190329,NaN,NaN,NaN,NaN,NaN,7280
14722,KOSDAQ,기타서비스,66201368625,037440,20190329,NaN,NaN,NaN,NaN,NaN,4755


##### 약 90개의 row수가 채워지고 나머지는 비어있음

# 값이 하나라도 있으면 fnguide 재무제표에서 가치지표 계산하기

In [1162]:
ticker = '102210'
url = f'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{ticker}'
response = requests.get(url)
df = pd.read_html(response.text, displayed_only = False)

equity = df[2][df[2]['IFRS(연결)'] == '자본']
income = df[0][df[0]['IFRS(연결)'] == '당기순이익']
cashflow = df[4][df[4]['IFRS(연결)'] == '영업활동으로인한현금흐름']
sales = df[0][df[0]['IFRS(연결)'] == '매출액']

C:\Users\user\AppData\Local\Temp\ipykernel_27820\3186047380.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text, displayed_only = False)


In [1163]:
def get_value_index_fnguide_seconde(row):
    
    if row.isna().sum() == 0:
        return row
    
    ticker = row['종목코드']
    date = row['날짜']
    cap = row['시가총액'] / 100000000

    try:
        url = f'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{ticker}'
        response = requests.get(url)
        df = pd.read_html(response.text, displayed_only = False)

        finan_date = date_dic[date]
    
        equity = df[2][df[2]['IFRS(연결)'] == '자본'] #57
        income = df[0][df[0]['IFRS(연결)'] == '당기순이익'] #76
        cashflow = df[4][df[4]['IFRS(연결)'] == '영업활동으로인한현금흐름'] #0
        sales = df[0][df[0]['IFRS(연결)'] == '매출액'] #0

        pbr = cap / equity.loc[57, finan_date] if not pd.isna(equity.loc[57, finan_date]) else np.nan
        per = cap / income.loc[76,finan_date] if not pd.isna(income.loc[76,finan_date]) else np.nan
        pcr = cap / cashflow.loc[0,finan_date] if not pd.isna(cashflow.loc[0,finan_date]) else np.nan
        psr = cap / sales.loc[0,finan_date] if not pd.isna(sales.loc[0,finan_date]) else np.nan
        
        row['PER'] = per if pd.isna(row['PER']) else np.nan
        row['PCR'] = pcr if pd.isna(row['PCR']) else np.nan
        row['PSR'] = psr if pd.isna(row['PSR']) else np.nan
        row['PBR'] = pbr if pd.isna(row['PBR']) else np.nan

        print(ticker,finan_date,pbr,per,pcr,psr)
        
    except Exception as e:
        return row
    
    return row
    

In [1164]:
temp = stocks.apply(get_value_index_fnguide_seconde,axis=1)

C:\Users\user\AppData\Local\Temp\ipykernel_27820\3087171448.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text, displayed_only = False)


475150 2023/12 nan nan nan nan


C:\Users\user\AppData\Local\Temp\ipykernel_27820\3087171448.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text, displayed_only = False)


030790 2023/12 nan nan nan nan


C:\Users\user\AppData\Local\Temp\ipykernel_27820\3087171448.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text, displayed_only = False)
C:\Users\user\AppData\Local\Temp\ipykernel_27820\3087171448.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text, displayed_only = False)
C:\Users\user\AppData\Local\Temp\ipykernel_27820\3087171448.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text, displayed_only = False)
C:\Users\user\AppData\Local\Temp\ipykernel_27820\3087171448.py:13: FutureWarning: Passing literal html to 'read_htm

...

030790 2021/12 6.934093758 -1.0137563973684212 -1.2471391651079138 1.3980027737903227


C:\Users\user\AppData\Local\Temp\ipykernel_27820\3087171448.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text, displayed_only = False)
C:\Users\user\AppData\Local\Temp\ipykernel_27820\3087171448.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text, displayed_only = False)


015540 2021/12 -42.53801947837838 -143.08242915454545 -20.440347022077923 16.394861673958335


C:\Users\user\AppData\Local\Temp\ipykernel_27820\3087171448.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text, displayed_only = False)
C:\Users\user\AppData\Local\Temp\ipykernel_27820\3087171448.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text, displayed_only = False)
C:\Users\user\AppData\Local\Temp\ipykernel_27820\3087171448.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text, displayed_only = False)
C:\Users\user\AppData\Local\Temp\ipykernel_27820\3087171448.py:13: FutureWarning: Passing literal html to 'read_htm

KeyboardInterrupt: 

In [986]:
temp

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR
0,KOSPI,서비스업,220154672535,095570,20240329,보통주,13.95137,0.53399,-12.708,0.23009
1,KOSPI,기타금융,200038171100,006840,20240329,보통주,6.57706,0.38567,0.42958,0.05057
2,KOSPI,기타금융,357980798340,027410,20240329,보통주,4.68667,0.22041,5.18776,0.82289
3,KOSPI,유통업,2034315736200,282330,20240329,보통주,11.58762,2.11104,3.5071,0.27693
4,KOSPI,기타금융,2502627163260,138930,20240329,보통주,3.62053,0.22491,-399.2222,0.23968
...,...,...,...,...,...,...,...,...,...,...
21572,KOSDAQ,유통,79200000000,024060,20200331,보통주,9.69508,1.01218,110.00543,0.53362
21573,KOSDAQ,기계·장비,34688389240,010240,20200331,보통주,7.78644,0.80795,5.05788,0.5219
21574,KOSDAQ,음식료·담배,39092374575,189980,20200331,보통주,12.31997,0.97553,6.63823,1.21519
21575,KOSDAQ,기타서비스,49285561500,037440,20200331,보통주,16.20406,1.01802,2.95334,0.3341


In [990]:
temp[temp.isna().any(axis=1)]

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR
159,KOSPI,건설업,328192582800,475150,20240329,보통주,NaN,NaN,NaN,NaN
380,KOSPI,기타금융,5578001344000,088980,20240329,보통주,NaN,NaN,NaN,NaN
381,KOSPI,기타금융,404757683160,094800,20240329,보통주,NaN,NaN,NaN,NaN
424,KOSPI,서비스업,19208323080,030790,20240329,보통주,NaN,NaN,NaN,NaN
809,KOSPI,기타금융,24366960000,152550,20240329,보통주,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
21537,KOSDAQ,금융,7059900000,279410,20200331,보통주,None,None,None,None
21538,KOSDAQ,운송장비·부품,81066045500,102210,20200331,보통주,-6.049705,NaN,-33.777519,NaN
21543,KOSDAQ,음식료·담배,80762168880,008800,20200331,보통주,NaN,3.22,NaN,12.64
21551,KOSDAQ,금속,23089235302,053660,20200331,보통주,-0.343079,-2.653935,-1.973439,NaN


In [1030]:
stocks = temp

In [1132]:
stocks

,시장구분,업종명,시가총액,종목코드,날짜,주식 종류,PER,PBR,PCR,PSR,종가
0,KOSPI,서비스업,220154672535,095570,20240329,보통주,13.95137,0.53399,-12.708,0.23009,4865
1,KOSPI,기타금융,200038171100,006840,20240329,보통주,6.57706,0.38567,0.42958,0.05057,15100
2,KOSPI,기타금융,357980798340,027410,20240329,보통주,4.68667,0.22041,5.18776,0.82289,3740
3,KOSPI,유통업,2034315736200,282330,20240329,보통주,11.58762,2.11104,3.5071,0.27693,117700
4,KOSPI,기타금융,2502627163260,138930,20240329,보통주,3.62053,0.22491,-399.2222,0.23968,7770
...,...,...,...,...,...,...,...,...,...,...,...
14719,KOSDAQ,유통,55425000000,024060,20190329,NaN,NaN,NaN,NaN,NaN,3695
14720,KOSDAQ,기계·장비,84410467600,010240,20190329,NaN,NaN,NaN,NaN,NaN,6850
14721,KOSDAQ,음식료·담배,53444598480,189980,20190329,NaN,NaN,NaN,NaN,NaN,7280
14722,KOSDAQ,기타서비스,66201368625,037440,20190329,NaN,NaN,NaN,NaN,NaN,4755


## 최종 데이터는 615개의 널값이 포함된 row수들임

# etf정보 변환

In [1133]:
etf_info[['종목코드', '날짜']] = etf_info['종목코드_날짜'].str.split('/',expand = True)

In [1134]:
etf_info = etf_info.drop(columns = ['종목코드_날짜'])

In [1135]:
filter = etf_info['날짜'] > '20181201'
etfs = etf_info[filter]

In [1136]:
etfs.groupby(['etf코드','날짜']).size().reset_index().shape

(1443, 3)

In [1137]:
etfs

,구성비중,etf코드,종목코드,날짜
0,31.12,KR7069500007,005930,20240329
1,8.22,KR7069500007,000660,20240329
2,2.61,KR7069500007,005380,20240329
3,2.57,KR7069500007,068270,20240329
4,2.29,KR7069500007,005490,20240329
...,...,...,...,...
139910,0.07,KR7316670009,108790,20190329
139911,1.40,KR7316670009,KRD010010001,20190329
139912,11.85,KR7319870002,292050,20190329
139913,2.06,KR7319870002,138230,20190329


#### 결론 1279개의 etf만 가치지표 사용가능

# 두 테이블 합치기

In [1494]:
info = etfs.merge(stocks,how = 'left',on = ['종목코드','날짜'])
info['종목 자산액'] = None

In [1496]:
info

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,종가,시가총액,주식 종류,PER,PBR,PCR,PSR,DPS,종목 자산액
0,31.12,KR7069500007,005930,20240329,KOSPI,전기전자,82400.0,4.919101e+14,보통주,None,None,None,None,None,None
1,8.22,KR7069500007,000660,20240329,KOSPI,전기전자,183000.0,1.332244e+14,보통주,None,None,None,None,None,None
2,2.61,KR7069500007,005380,20240329,KOSPI,운수장비,233000.0,4.928684e+13,보통주,None,None,None,None,None,None
3,2.57,KR7069500007,068270,20240329,KOSPI,의약품,191200.0,4.169111e+13,보통주,None,None,None,None,None,None
4,2.29,KR7069500007,005490,20240329,KOSPI,철강금속,422000.0,3.568906e+13,보통주,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139910,0.07,KR7316670009,108790,20190329,KOSDAQ,유통,5350.0,1.773818e+11,보통주,None,None,None,None,None,None
139911,1.40,KR7316670009,KRD010010001,20190329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
139912,11.85,KR7319870002,292050,20190329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
139913,2.06,KR7319870002,138230,20190329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [1497]:
print(info['날짜'].dtype)
print(etf_price['날짜'].dtype)

object
int64


In [1498]:
filt = info['날짜'].isna()
info[filt]

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,종가,시가총액,주식 종류,PER,PBR,PCR,PSR,DPS,종목 자산액


In [1499]:
info['날짜'] = info['날짜'].astype('int')

In [1500]:
print(etf_price['단축코드'].dtype)
print(info['종목코드'].dtype)

object
object


In [1501]:
etf_price['단축코드'] = etf_price['단축코드'].astype('str').str.zfill(6)

In [1502]:
filter = info['시장구분'].isna()

In [1503]:
info[filter]['종목코드'].value_counts()

종목코드
KRD010010001    1431
069500            13
102110            12
310960            10
292050             6
137930             5
138230             5
229200             4
114800             3
091170             3
404120             2
441800             1
251340             1
228810             1
Name: count, dtype: int64

In [1504]:
info[filter]['etf코드'].value_counts()

etf코드
KR7166400002    22
KR7319870002    17
KR7289480006    16
KR7360150007    11
KR7360140008     9
                ..
KR7465330009     1
KR7466810009     1
KR7466920006     1
KR7466930005     1
KR7451060008     1
Name: count, Length: 335, dtype: int64

In [1505]:
filter = info['종목코드'] == 'KRD010010001'
info[filter]['구성비중'].value_counts()

구성비중
100.00    17
0.51      12
0.40      10
0.38      10
0.55      10
          ..
3.97       1
2.08       1
5.90       1
5.56       1
86.09      1
Name: count, Length: 477, dtype: int64

### 현금정보 가져오고, info 날짜 타입 etf_price 날짜 타입과 맞추기

In [1512]:
etf_price

,단축코드,시가총액,날짜,표준코드
0,152100,827604000000,20190329,KR7152100004
1,295820,6338500000,20190329,KR7295820005
2,253150,7980000000,20190329,KR7253150007
3,253160,3678750000,20190329,KR7253160006
4,292750,37990000000,20190329,KR7292750007
...,...,...,...,...
3209,429870,4875000000,20240329,KR7429870009
3210,476450,380127550000,20240329,KR7476450002
3211,460270,8930250000,20240329,KR7460270002
3212,459790,9360000000,20240329,KR7459790002


In [1513]:
info['구성비중'].dtype

dtype('float64')

In [1524]:
def get_etf_price(code, date, weight):
    filtered_df = etf_price[(etf_price['표준코드'] == code) & (etf_price['날짜'] == date)]
    value =  (filtered_df['시가총액'].values[0]) * (weight*100)
    return value

In [1525]:

for index, row in info.iterrows():
    # print(row[['종목코드','구성비중']])
    if row['종목코드'] == 'KRD010010001':
        info.at[index, '시장구분'] = '현금'
        info.at[index, '업종명'] = '현금'
        info.at[index, '주식 종류'] = '현금'
        info.at[index, '종가'] = 1
        info.at[index, 'PER'] = 0
        info.at[index, 'PBR'] = 0
        info.at[index, 'PCR'] = 0
        info.at[index, 'PSR'] = 0
        try:
            price = get_etf_price(row['etf코드'], row['날짜'], row['구성비중'])
            info.at[index, '시가총액'] = price
            info.at[index,'종목 자산액'] = price
        except Exception as e:
            print(row['etf코드'])
    
    else:
        info.at[index,'종목 자산액'] = get_etf_price(row['etf코드'], row['날짜'], row['구성비중'])
        

KR7069500007 20240329 7392660000000 31.12 23005957920000000.0
KR7069500007 20240329 7392660000000 8.22 6076766520000000.739266000000
KR7069500007 20240329 7392660000000 2.61 1929484260000000.0
KR7069500007 20240329 7392660000000 2.57 1899913620000000.0
KR7069500007 20240329 7392660000000 2.29 1692919140000000.0
KR7069500007 20240329 7392660000000 2.2 1626385200000000.221779800000
KR7069500007 20240329 7392660000000 2.0 1478532000000000.0
KR7069500007 20240329 7392660000000 1.96 1448961360000000.0
KR7069500007 20240329 7392660000000 1.91 1411998060000000.0
KR7069500007 20240329 7392660000000 1.63 1205003580000000.0
KR7069500007 20240329 7392660000000 1.62 1197610920000000.0
KR7069500007 20240329 7392660000000 1.36 1005401760000000.0
KR7069500007 20240329 7392660000000 1.3 961045800000000.0
KR7069500007 20240329 7392660000000 1.25 924082500000000.0
KR7069500007 20240329 7392660000000 1.22 901904520000000.0
KR7069500007 20240329 7392660000000 1.1 813192600000000.0739266000000
KR7069500007

C:\Users\user\AppData\Local\Temp\ipykernel_27820\2626701524.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '916689840000000.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  info.at[index, '시가총액'] = price


KR7069660009 20240329 354748500000 0.18 6385473000000.0
KR7069660009 20240329 354748500000 0.18 6385473000000.0
KR7069660009 20240329 354748500000 0.18 6385473000000.0
KR7069660009 20240329 354748500000 0.17 6030724500000.0
KR7069660009 20240329 354748500000 0.17 6030724500000.0
KR7069660009 20240329 354748500000 0.17 6030724500000.0
KR7069660009 20240329 354748500000 0.16 5675976000000.0
KR7069660009 20240329 354748500000 0.16 5675976000000.0
KR7069660009 20240329 354748500000 0.16 5675976000000.0
KR7069660009 20240329 354748500000 0.15 5321227500000.0
KR7069660009 20240329 354748500000 0.15 5321227500000.0
KR7069660009 20240329 354748500000 0.15 5321227500000.0
KR7069660009 20240329 354748500000 0.15 5321227500000.0
KR7069660009 20240329 354748500000 0.14 4966479000000.000709497000000
KR7069660009 20240329 354748500000 0.14 4966479000000.000709497000000
KR7069660009 20240329 354748500000 0.14 4966479000000.000709497000000
KR7069660009 20240329 354748500000 0.14 4966479000000.00070949

KeyboardInterrupt: 

In [1191]:
info

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,시가총액,주식 종류,PER,PBR,PCR,PSR,종가,종목 자산액
0,31.12,KR7069500007,005930,20240329,KOSPI,전기전자,4.919101e+14,보통주,36.84169,1.50955,12.08101,2.05929,82400.0,2300595792000.0
1,8.22,KR7069500007,000660,20240329,KOSPI,전기전자,1.332244e+14,보통주,-11.3046,1.81988,24.07848,3.14391,183000.0,607676652000.000122
2,2.61,KR7069500007,005380,20240329,KOSPI,운수장비,4.928684e+13,보통주,4.6686,0.57835,-22.17142,0.34331,233000.0,192948426000.0
3,2.57,KR7069500007,068270,20240329,KOSPI,의약품,4.169111e+13,보통주,55.3685,2.50701,55.21279,13.62691,191200.0,189991361999.999969
4,2.29,KR7069500007,005490,20240329,KOSPI,철강금속,3.568906e+13,보통주,24.87694,0.69951,6.84913,0.54771,422000.0,169291914000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139910,0.07,KR7316670009,108790,20190329,KOSDAQ,유통,1.773818e+11,NaN,NaN,NaN,NaN,NaN,5350.0,25573100.0
139911,1.40,KR7316670009,KRD010010001,20190329,현금,현금,5.114620e+08,현금,0,0,0,0,NaN,511462000.0
139912,11.85,KR7319870002,292050,20190329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,822034500.0
139913,2.06,KR7319870002,138230,20190329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,142902200.0


In [1192]:
filter = info['시장구분'].isna()
info[filter]['종목코드'].value_counts()

종목코드
069500    13
102110    12
310960    10
292050     6
137930     5
138230     5
229200     4
114800     3
091170     3
404120     2
441800     1
251340     1
228810     1
Name: count, dtype: int64

In [1193]:
info[filter]['etf코드'].value_counts()

etf코드
KR7166400002    16
KR7319870002    11
KR7289480006    10
KR7360150007     7
KR7360140008     5
KR7385720008     4
KR7402520001     3
KR7442260006     2
KR7403790009     1
KR7472150002     1
KR7159800002     1
KR7438740003     1
KR7105190003     1
KR7305050007     1
KR7332500008     1
KR7410870000     1
Name: count, dtype: int64

In [1194]:
info

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,시가총액,주식 종류,PER,PBR,PCR,PSR,종가,종목 자산액
0,31.12,KR7069500007,005930,20240329,KOSPI,전기전자,4.919101e+14,보통주,36.84169,1.50955,12.08101,2.05929,82400.0,2300595792000.0
1,8.22,KR7069500007,000660,20240329,KOSPI,전기전자,1.332244e+14,보통주,-11.3046,1.81988,24.07848,3.14391,183000.0,607676652000.000122
2,2.61,KR7069500007,005380,20240329,KOSPI,운수장비,4.928684e+13,보통주,4.6686,0.57835,-22.17142,0.34331,233000.0,192948426000.0
3,2.57,KR7069500007,068270,20240329,KOSPI,의약품,4.169111e+13,보통주,55.3685,2.50701,55.21279,13.62691,191200.0,189991361999.999969
4,2.29,KR7069500007,005490,20240329,KOSPI,철강금속,3.568906e+13,보통주,24.87694,0.69951,6.84913,0.54771,422000.0,169291914000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139910,0.07,KR7316670009,108790,20190329,KOSDAQ,유통,1.773818e+11,NaN,NaN,NaN,NaN,NaN,5350.0,25573100.0
139911,1.40,KR7316670009,KRD010010001,20190329,현금,현금,5.114620e+08,현금,0,0,0,0,NaN,511462000.0
139912,11.85,KR7319870002,292050,20190329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,822034500.0
139913,2.06,KR7319870002,138230,20190329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,142902200.0


In [1195]:
for index, row in info.iterrows():
    # print(row[['종목코드','구성비중']])
    if pd.isna(row['시장구분']) and row['종목코드'] in etf_price['단축코드'].values:

        filt = etf_price['단축코드'] == row['종목코드']
        filt2 = etf_price['날짜'] == row['날짜']
        
        # print(etf_price[filt & filt2]['시가총액'].values[0])
        
        info.at[index, '시장구분'] = 'etf'
        info.at[index, '업종명'] = 'etf'
        info.at[index, '시가총액'] = etf_price[filt & filt2]['시가총액'].values[0]
        

In [1196]:
filter = info['시장구분'].isna()
info[filter]['종목코드'].value_counts()

종목코드
137930    5
Name: count, dtype: int64

In [1197]:
info[filter]['etf코드'].value_counts()

etf코드
KR7166400002    5
Name: count, dtype: int64

In [1198]:
info

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,시가총액,주식 종류,PER,PBR,PCR,PSR,종가,종목 자산액
0,31.12,KR7069500007,005930,20240329,KOSPI,전기전자,4.919101e+14,보통주,36.84169,1.50955,12.08101,2.05929,82400.0,2300595792000.0
1,8.22,KR7069500007,000660,20240329,KOSPI,전기전자,1.332244e+14,보통주,-11.3046,1.81988,24.07848,3.14391,183000.0,607676652000.000122
2,2.61,KR7069500007,005380,20240329,KOSPI,운수장비,4.928684e+13,보통주,4.6686,0.57835,-22.17142,0.34331,233000.0,192948426000.0
3,2.57,KR7069500007,068270,20240329,KOSPI,의약품,4.169111e+13,보통주,55.3685,2.50701,55.21279,13.62691,191200.0,189991361999.999969
4,2.29,KR7069500007,005490,20240329,KOSPI,철강금속,3.568906e+13,보통주,24.87694,0.69951,6.84913,0.54771,422000.0,169291914000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139910,0.07,KR7316670009,108790,20190329,KOSDAQ,유통,1.773818e+11,NaN,NaN,NaN,NaN,NaN,5350.0,25573100.0
139911,1.40,KR7316670009,KRD010010001,20190329,현금,현금,5.114620e+08,현금,0,0,0,0,NaN,511462000.0
139912,11.85,KR7319870002,292050,20190329,etf,etf,8.436600e+10,NaN,NaN,NaN,NaN,NaN,NaN,822034500.0
139913,2.06,KR7319870002,138230,20190329,etf,etf,4.485465e+10,NaN,NaN,NaN,NaN,NaN,NaN,142902200.0


In [1199]:
info_final = info[info['etf코드'] != 'KR7166400002']

info_final

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,시가총액,주식 종류,PER,PBR,PCR,PSR,종가,종목 자산액
0,31.12,KR7069500007,005930,20240329,KOSPI,전기전자,4.919101e+14,보통주,36.84169,1.50955,12.08101,2.05929,82400.0,2300595792000.0
1,8.22,KR7069500007,000660,20240329,KOSPI,전기전자,1.332244e+14,보통주,-11.3046,1.81988,24.07848,3.14391,183000.0,607676652000.000122
2,2.61,KR7069500007,005380,20240329,KOSPI,운수장비,4.928684e+13,보통주,4.6686,0.57835,-22.17142,0.34331,233000.0,192948426000.0
3,2.57,KR7069500007,068270,20240329,KOSPI,의약품,4.169111e+13,보통주,55.3685,2.50701,55.21279,13.62691,191200.0,189991361999.999969
4,2.29,KR7069500007,005490,20240329,KOSPI,철강금속,3.568906e+13,보통주,24.87694,0.69951,6.84913,0.54771,422000.0,169291914000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139910,0.07,KR7316670009,108790,20190329,KOSDAQ,유통,1.773818e+11,NaN,NaN,NaN,NaN,NaN,5350.0,25573100.0
139911,1.40,KR7316670009,KRD010010001,20190329,현금,현금,5.114620e+08,현금,0,0,0,0,NaN,511462000.0
139912,11.85,KR7319870002,292050,20190329,etf,etf,8.436600e+10,NaN,NaN,NaN,NaN,NaN,NaN,822034500.0
139913,2.06,KR7319870002,138230,20190329,etf,etf,4.485465e+10,NaN,NaN,NaN,NaN,NaN,NaN,142902200.0


In [1200]:
info[info['etf코드'] == 'KR7166400002']

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,시가총액,주식 종류,PER,PBR,PCR,PSR,종가,종목 자산액
3429,26.03,KR7166400002,005930,20240329,KOSPI,전기전자,4.919101e+14,보통주,36.84169,1.50955,12.08101,2.05929,82400.0,1777849000.0
3430,7.92,KR7166400002,102110,20240329,etf,etf,2.585700e+12,NaN,NaN,NaN,NaN,NaN,NaN,540936000.0
3431,6.87,KR7166400002,000660,20240329,KOSPI,전기전자,1.332244e+14,보통주,-11.3046,1.81988,24.07848,3.14391,183000.0,469221000.0
3432,2.18,KR7166400002,005380,20240329,KOSPI,운수장비,4.928684e+13,보통주,4.6686,0.57835,-22.17142,0.34331,233000.0,148894000.0
3433,2.18,KR7166400002,310960,20240329,etf,etf,2.127500e+10,NaN,NaN,NaN,NaN,NaN,NaN,148894000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127562,0.01,KR7166400002,036580,20190329,KOSPI,음식료품,2.978636e+11,NaN,NaN,NaN,NaN,NaN,8110.0,857150.0
127563,0.01,KR7166400002,192400,20190329,KOSPI,기타금융,9.992974e+11,NaN,NaN,NaN,NaN,NaN,140500.0,857150.0
127564,0.01,KR7166400002,003300,20190329,KOSPI,서비스업,3.279727e+11,NaN,NaN,NaN,NaN,NaN,54700.0,857150.0
127565,0.01,KR7166400002,097230,20190329,KOSPI,건설업,1.262025e+11,NaN,NaN,NaN,NaN,NaN,1190.0,857150.0


# 최종 정보 저장하기

In [1230]:
info_final

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,시가총액,주식 종류,PER,PBR,PCR,PSR,종가,종목 자산액
0,31.12,KR7069500007,005930,20240329,KOSPI,전기전자,4.919101e+14,보통주,36.84169,1.50955,12.08101,2.05929,82400.0,2300595792000.0
1,8.22,KR7069500007,000660,20240329,KOSPI,전기전자,1.332244e+14,보통주,-11.3046,1.81988,24.07848,3.14391,183000.0,607676652000.000122
2,2.61,KR7069500007,005380,20240329,KOSPI,운수장비,4.928684e+13,보통주,4.6686,0.57835,-22.17142,0.34331,233000.0,192948426000.0
3,2.57,KR7069500007,068270,20240329,KOSPI,의약품,4.169111e+13,보통주,55.3685,2.50701,55.21279,13.62691,191200.0,189991361999.999969
4,2.29,KR7069500007,005490,20240329,KOSPI,철강금속,3.568906e+13,보통주,24.87694,0.69951,6.84913,0.54771,422000.0,169291914000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139910,0.07,KR7316670009,108790,20190329,KOSDAQ,유통,1.773818e+11,NaN,NaN,NaN,NaN,NaN,5350.0,25573100.0
139911,1.40,KR7316670009,KRD010010001,20190329,현금,현금,5.114620e+08,현금,0,0,0,0,1.0,511462000.0
139912,11.85,KR7319870002,292050,20190329,etf,etf,8.436600e+10,NaN,NaN,NaN,NaN,NaN,NaN,822034500.0
139913,2.06,KR7319870002,138230,20190329,etf,etf,4.485465e+10,NaN,NaN,NaN,NaN,NaN,NaN,142902200.0


In [1232]:
info_final.to_csv('etf_with_stock.csv',index = False)

# null값을 가지고 있는 row삭제하기

In [1236]:
index = info_final[(info_final['날짜'] != 20190329) & (info_final.isna().any(axis=1))].index
temp = info_final.drop(index = index)

In [1238]:
temp[temp['날짜'] > 20190329].isna().sum()

구성비중      0
etf코드     0
종목코드      0
날짜        0
시장구분      0
업종명       0
시가총액      0
주식 종류     0
PER       0
PBR       0
PCR       0
PSR       0
종가        0
종목 자산액    0
dtype: int64

In [1239]:
info_final = temp

In [1310]:
info_final

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,시가총액,주식 종류,PER,PBR,PCR,PSR,종가,종목 자산액
0,31.12,KR7069500007,005930,20240329,KOSPI,전기전자,4.919101e+14,보통주,36.84169,1.50955,12.08101,2.05929,82400.0,2300595792000.0
1,8.22,KR7069500007,000660,20240329,KOSPI,전기전자,1.332244e+14,보통주,-11.3046,1.81988,24.07848,3.14391,183000.0,607676652000.000122
2,2.61,KR7069500007,005380,20240329,KOSPI,운수장비,4.928684e+13,보통주,4.6686,0.57835,-22.17142,0.34331,233000.0,192948426000.0
3,2.57,KR7069500007,068270,20240329,KOSPI,의약품,4.169111e+13,보통주,55.3685,2.50701,55.21279,13.62691,191200.0,189991361999.999969
4,2.29,KR7069500007,005490,20240329,KOSPI,철강금속,3.568906e+13,보통주,24.87694,0.69951,6.84913,0.54771,422000.0,169291914000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139910,0.07,KR7316670009,108790,20190329,KOSDAQ,유통,1.773818e+11,NaN,NaN,NaN,NaN,NaN,5350.0,25573100.0
139911,1.40,KR7316670009,KRD010010001,20190329,현금,현금,5.114620e+08,현금,0,0,0,0,1.0,511462000.0
139912,11.85,KR7319870002,292050,20190329,etf,etf,8.436600e+10,NaN,NaN,NaN,NaN,NaN,NaN,822034500.0
139913,2.06,KR7319870002,138230,20190329,etf,etf,4.485465e+10,NaN,NaN,NaN,NaN,NaN,NaN,142902200.0


In [1311]:
info_final.isna().sum()

구성비중          0
etf코드         0
종목코드          0
날짜            0
시장구분          0
업종명           0
시가총액          0
주식 종류     15578
PER       15578
PBR       15578
PCR       15578
PSR       15578
종가            4
종목 자산액        0
dtype: int64

In [1312]:
info_final[(info_final['날짜'] == 20190329) & (info_final['종가'].isna())]

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,시가총액,주식 종류,PER,PBR,PCR,PSR,종가,종목 자산액
134374,21.19,KR7289480006,310960,20190329,etf,etf,8.051250e+10,NaN,NaN,NaN,NaN,NaN,NaN,2476687200.0
134422,0.29,KR7289480006,102110,20190329,etf,etf,4.160570e+12,NaN,NaN,NaN,NaN,NaN,NaN,33895200.0
139912,11.85,KR7319870002,292050,20190329,etf,etf,8.436600e+10,NaN,NaN,NaN,NaN,NaN,NaN,822034500.0
139913,2.06,KR7319870002,138230,20190329,etf,etf,4.485465e+10,NaN,NaN,NaN,NaN,NaN,NaN,142902200.0


In [1313]:
info_final[(info_final['날짜'] == 20190329) & (info_final['업종명'] == 'etf')]

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,시가총액,주식 종류,PER,PBR,PCR,PSR,종가,종목 자산액
134374,21.19,KR7289480006,310960,20190329,etf,etf,8.051250e+10,NaN,NaN,NaN,NaN,NaN,NaN,2476687200.0
134422,0.29,KR7289480006,102110,20190329,etf,etf,4.160570e+12,NaN,NaN,NaN,NaN,NaN,NaN,33895200.0
139912,11.85,KR7319870002,292050,20190329,etf,etf,8.436600e+10,NaN,NaN,NaN,NaN,NaN,NaN,822034500.0
139913,2.06,KR7319870002,138230,20190329,etf,etf,4.485465e+10,NaN,NaN,NaN,NaN,NaN,NaN,142902200.0


In [1316]:
index = info_final[(info_final['업종명'] == 'etf')].index
info_final = info_final.drop(index = index)

In [1367]:
info_final

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,시가총액,주식 종류,PER,PBR,PCR,PSR,종가,종목 자산액
0,31.12,KR7069500007,005930,20240329,KOSPI,전기전자,4.919101e+14,보통주,36.84169,1.50955,12.08101,2.05929,82400.0,2300595792000.0
1,8.22,KR7069500007,000660,20240329,KOSPI,전기전자,1.332244e+14,보통주,-11.3046,1.81988,24.07848,3.14391,183000.0,607676652000.000122
2,2.61,KR7069500007,005380,20240329,KOSPI,운수장비,4.928684e+13,보통주,4.6686,0.57835,-22.17142,0.34331,233000.0,192948426000.0
3,2.57,KR7069500007,068270,20240329,KOSPI,의약품,4.169111e+13,보통주,55.3685,2.50701,55.21279,13.62691,191200.0,189991361999.999969
4,2.29,KR7069500007,005490,20240329,KOSPI,철강금속,3.568906e+13,보통주,24.87694,0.69951,6.84913,0.54771,422000.0,169291914000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139908,0.09,KR7316670009,121440,20190329,KOSDAQ,금융,1.499289e+11,NaN,NaN,NaN,NaN,NaN,3500.0,32879700.0
139909,0.08,KR7316670009,025900,20190329,KOSDAQ,종이·목재,2.641643e+11,NaN,NaN,NaN,NaN,NaN,18400.0,29226400.0
139910,0.07,KR7316670009,108790,20190329,KOSDAQ,유통,1.773818e+11,NaN,NaN,NaN,NaN,NaN,5350.0,25573100.0
139911,1.40,KR7316670009,KRD010010001,20190329,현금,현금,5.114620e+08,현금,0,0,0,0,1.0,511462000.0


In [1317]:
info_final[['etf코드' , '날짜','종목 자산액' , '구성비중']].groupby(['etf코드', '날짜']).sum()

종목 자산액    구성비중
etf코드        날짜                               
KR7069500007 20190329  6980356250000.0  100.00
             20200331  5868592044500.0   99.80
             20210331  4695825384250.0   99.70
             20220331  4974225453000.0   99.85
             20230331  5743177791750.0   99.85
...                                ...     ...
KR7475720009 20240329    31619011450.0   99.97
KR7476000005 20240329     6541920000.0   94.40
KR7476260005 20240329     4470447000.0  100.01
KR7476850003 20240329   137150174550.0  100.02
KR7477080006 20240329   170119000000.0  100.00

[1437 rows x 2 columns]

# 로그 수익률 구하기

In [1336]:
columns = ['구성비중','etf코드' , '종목 자산액','시가총액']
etf_stocks = info_final.drop(columns = columns).drop_duplicates()

In [1350]:
etf_stocks['로그 수익률'] = None

In [1351]:
etf_stocks[etf_stocks['종목코드'] == '000660']

,종목코드,날짜,시장구분,업종명,주식 종류,PER,PBR,PCR,PSR,종가,로그 수익률
1,000660,20240329,KOSPI,전기전자,보통주,-11.3046,1.81988,24.07848,3.14391,183000.0,None
28333,000660,20230331,KOSPI,전기전자,보통주,24.48922,0.81519,3.69406,1.22363,88600.0,None
55483,000660,20220331,KOSPI,전기전자,보통주,9.9318,1.4492,4.81715,2.21798,118000.0,None
81262,000660,20210331,KOSPI,전기전자,보통주,18.14226,1.56208,7.00538,2.7043,132500.0,None
104511,000660,20200331,KOSPI,전기전자,보통주,34.15049,1.34314,10.4592,2.53809,83300.0,None
123971,000660,20190329,KOSPI,전기전자,NaN,NaN,NaN,NaN,NaN,74200.0,None


In [1359]:
date_dic = {
    20240329 : 20230331,
    20230331 : 20220331,
    20220331 : 20210331,
    20210331 : 20200331,
    20200331 : 20190329
}

In [1360]:
def get_log_earning_rate(row):
    if row['날짜'] == 20190329:
        return None

    date = date_dic[row['날짜']]
    ticker = row['종목코드']
    price = row['종가']

    try:
        prev_price = etf_stocks[(etf_stocks['날짜'] == date) & (etf_stocks['종목코드'] == ticker)]['종가'].values[0]

        earning_rate = np.log(price/prev_price)
        
        return earning_rate
    except Exception as e:
        return None

In [1361]:
for index, row in etf_stocks.iterrows():
    log_earning_rate = get_log_earning_rate(row)
    etf_stocks.at[index,'로그 수익률'] = log_earning_rate

In [1362]:
etf_stocks

,종목코드,날짜,시장구분,업종명,주식 종류,PER,PBR,PCR,PSR,종가,로그 수익률
0,005930,20240329,KOSPI,전기전자,보통주,36.84169,1.50955,12.08101,2.05929,82400.0,0.252702
1,000660,20240329,KOSPI,전기전자,보통주,-11.3046,1.81988,24.07848,3.14391,183000.0,0.725354
2,005380,20240329,KOSPI,운수장비,보통주,4.6686,0.57835,-22.17142,0.34331,233000.0,0.233389
3,068270,20240329,KOSPI,의약품,보통주,55.3685,2.50701,55.21279,13.62691,191200.0,0.244019
4,005490,20240329,KOSPI,철강금속,보통주,24.87694,0.69951,6.84913,0.54771,422000.0,0.136922
...,...,...,...,...,...,...,...,...,...,...,...
139706,072020,20190329,KOSDAQ,제약,NaN,NaN,NaN,NaN,NaN,18250.0,None
139707,005860,20190329,KOSDAQ,음식료·담배,NaN,NaN,NaN,NaN,NaN,1860.0,None
139708,012790,20190329,KOSDAQ,제약,NaN,NaN,NaN,NaN,NaN,10300.0,None
139709,052670,20190329,KOSDAQ,제약,NaN,NaN,NaN,NaN,NaN,3850.0,None


In [1363]:
etf_stocks[etf_stocks['종목코드'] == '005930']

,종목코드,날짜,시장구분,업종명,주식 종류,PER,PBR,PCR,PSR,종가,로그 수익률
0,005930,20240329,KOSPI,전기전자,보통주,36.84169,1.50955,12.08101,2.05929,82400.0,0.252702
28332,005930,20230331,KOSPI,전기전자,보통주,6.86341,1.08821,6.04095,1.24287,64000.0,-0.083881
55482,005930,20220331,KOSPI,전기전자,보통주,13.55287,1.7954,8.1693,1.90221,69600.0,-0.156611
81261,005930,20210331,KOSPI,전기전자,보통주,21.08809,2.05554,8.4275,2.32344,81400.0,0.533396
104510,005930,20200331,KOSPI,전기전자,보통주,17.6252,1.48689,8.35184,1.64509,47750.0,0.067125
123970,005930,20190329,KOSPI,전기전자,NaN,NaN,NaN,NaN,NaN,44650.0,None


In [1365]:
info_final[['etf코드' , '날짜','종목 자산액' , '구성비중']].groupby(['etf코드', '날짜']).sum()

종목 자산액    구성비중
etf코드        날짜                               
KR7069500007 20190329  6980356250000.0  100.00
             20200331  5868592044500.0   99.80
             20210331  4695825384250.0   99.70
             20220331  4974225453000.0   99.85
             20230331  5743177791750.0   99.85
...                                ...     ...
KR7475720009 20240329    31619011450.0   99.97
KR7476000005 20240329     6541920000.0   94.40
KR7476260005 20240329     4470447000.0  100.01
KR7476850003 20240329   137150174550.0  100.02
KR7477080006 20240329   170119000000.0  100.00

[1437 rows x 2 columns]

In [1368]:
temp = pd.merge(info_final,etf_stocks,how = 'left')

In [1375]:
temp[temp['날짜'] > 20190329].isna().sum()

구성비중         0
etf코드        0
종목코드         0
날짜           0
시장구분         0
업종명          0
시가총액         0
주식 종류        0
PER          0
PBR          0
PCR          0
PSR          0
종가           0
종목 자산액       0
로그 수익률    3886
dtype: int64

In [1389]:
temp[(temp['날짜'] > 20190329) & (temp['로그 수익률'] == 0) & (temp['시장구분'] != '현금')]['로그 수익률'].unique()

array([0.0], dtype=object)

In [1390]:
temp[(temp['날짜'] > 20190329) & (temp['로그 수익률'] == 0) & (temp['시장구분'] != '현금')]

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,시가총액,주식 종류,PER,PBR,PCR,PSR,종가,종목 자산액,로그 수익률
2196,0.17,KR7143860005,268600,20240329,KOSDAQ,기타서비스,2.449412e+11,보통주,-7.54218,-10.04405,-27.94551,13.02319,6680.0,435499200.0,0.0
4279,0.02,KR7226490001,030610,20240329,KOSPI,증권,5.789318e+11,보통주,5.96455,0.29761,-0.27055,0.10766,5080.0,66654900.0,0.0
5007,0.03,KR7227830007,030610,20240329,KOSPI,증권,5.789318e+11,보통주,5.96455,0.29761,-0.27055,0.10766,5080.0,2170500.0,0.0
5557,0.07,KR7229200001,268600,20240329,KOSDAQ,기타서비스,2.449412e+11,보통주,-7.54218,-10.04405,-27.94551,13.02319,6680.0,617494850.0,0.0
5948,0.89,KR7244580007,268600,20240329,KOSDAQ,기타서비스,2.449412e+11,보통주,-7.54218,-10.04405,-27.94551,13.02319,6680.0,1096928560.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114858,0.03,KR7292750007,214370,20200331,KOSDAQ,제약,8.218395e+11,보통주,29.27875,3.50966,11.79795,12.7026,76500.0,5352000.0,0.0
115837,0.46,KR7301400008,214370,20200331,KOSDAQ,제약,8.218395e+11,보통주,29.27875,3.50966,11.79795,12.7026,76500.0,19116450.0,0.0
116763,0.03,KR7304760002,214370,20200331,KOSDAQ,제약,8.218395e+11,보통주,29.27875,3.50966,11.79795,12.7026,76500.0,5115000.0,0.0
116865,0.50,KR7304770001,214370,20200331,KOSDAQ,제약,8.218395e+11,보통주,29.27875,3.50966,11.79795,12.7026,76500.0,148411500.0,0.0


In [1393]:
temp[(temp['날짜'] > 20190329) & (temp['로그 수익률'] == 0) & (temp['시장구분'] != '현금')]['종목코드'].unique()

array(['268600', '030610', '078130', '265520', '181340', '017890',
       '007070', '057880', '064350', '001800', '001430', '215600',
       '007160', '003620', '004650', '041930', '214370', '051360',
       '180400'], dtype=object)

In [1417]:
etf_stocks[etf_stocks['종목코드'] == '180400']

,종목코드,날짜,시장구분,업종명,주식 종류,PER,PBR,PCR,PSR,종가,로그 수익률
15115,180400,20240329,KOSDAQ,기타서비스,보통주,-6.36058,8.43464,-14.49287,3.73951,4995.0,-0.252537
43490,180400,20230331,KOSDAQ,기타서비스,보통주,-284.85113,6.53661,256.5812,6.90033,6430.0,-0.185427
70980,180400,20220331,KOSDAQ,기타서비스,보통주,-26.01386,6.26667,-68.52345,25.31384,7740.0,None
119737,180400,20200331,KOSDAQ,기타서비스,보통주,-10.31468,7.26141,-42.49673,30.63304,7740.0,0.0
139319,180400,20190329,KOSDAQ,기타서비스,NaN,NaN,NaN,NaN,NaN,7740.0,None


#### 003620, 004650, 214370  데이터 이상함

# 현금과 none값을 0으로 대체

In [1424]:
index = temp[temp['주식 종류'] == '현금']['로그 수익률'].index
temp.loc[index,'로그 수익률'] = 0

In [1430]:
index = temp[(temp['날짜'] > 20190329) & (temp['로그 수익률'].isna())].index
temp.loc[index,'로그 수익률'] = 0

In [1435]:
info_final = temp[temp['날짜'] > 20190329]

In [1436]:
info_final

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,시가총액,주식 종류,PER,PBR,PCR,PSR,종가,종목 자산액,로그 수익률
0,31.12,KR7069500007,005930,20240329,KOSPI,전기전자,4.919101e+14,보통주,36.84169,1.50955,12.08101,2.05929,82400.0,2300595792000.0,0.252702
1,8.22,KR7069500007,000660,20240329,KOSPI,전기전자,1.332244e+14,보통주,-11.3046,1.81988,24.07848,3.14391,183000.0,607676652000.000122,0.725354
2,2.61,KR7069500007,005380,20240329,KOSPI,운수장비,4.928684e+13,보통주,4.6686,0.57835,-22.17142,0.34331,233000.0,192948426000.0,0.233389
3,2.57,KR7069500007,068270,20240329,KOSPI,의약품,4.169111e+13,보통주,55.3685,2.50701,55.21279,13.62691,191200.0,189991361999.999969,0.244019
4,2.29,KR7069500007,005490,20240329,KOSPI,철강금속,3.568906e+13,보통주,24.87694,0.69951,6.84913,0.54771,422000.0,169291914000.0,0.136922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121797,0.04,KR7337160006,002270,20200331,KOSPI,음식료품,3.395610e+11,보통주,12.46,0.55,5.26,0.26,300000.0,2641600.0,-0.749843
121798,0.03,KR7337160006,071840,20200331,KOSPI,유통업,3.694607e+11,보통주,-7.30155,0.37956,5.59604,0.18117,15650.0,1981200.0,-1.185269
121799,0.02,KR7337160006,019680,20200331,KOSPI,서비스업,5.005938e+11,보통주,37.64045,0.86124,8.45308,0.83365,5910.0,1320800.0,-0.046289
121800,0.02,KR7337160006,079430,20200331,KOSPI,기타제조업,1.458005e+11,보통주,14.08357,0.58636,2.86629,0.22152,7100.0,1320800.0,-1.062766


In [1440]:
info_final[['날짜', 'etf코드']].drop_duplicates()

,날짜,etf코드
0,20240329,KR7069500007
201,20240329,KR7069660009
390,20240329,KR7091160002
441,20240329,KR7091170001
452,20240329,KR7091180000
...,...,...
121067,20200331,KR7333980001
121156,20200331,KR7337120000
121401,20200331,KR7337140008
121500,20200331,KR7337150007


In [1441]:
info_final.to_csv('etf_with_stock.csv',index = False)

# 베타값 구하기

In [1527]:
import math

In [1546]:
date_dic = {
    20240329 : 20230331,
    20230331 : 20220331,
    20220331 : 20210331,
    20210331 : 20200331,
    20200331 : 20190329
}

In [1547]:
etf_combination = info_final[['etf코드','날짜']].drop_duplicates().values.tolist()
columns = ['etf코드' , '날짜', '수익률']
etf_earning = pd.DataFrame(columns = columns)

In [1549]:
for etf_com in etf_combination:
    ticker = etf_com[0]
    date = etf_com[1]
    prev_date = date_dic[date]

    filt1 = info_final['etf코드'] == ticker
    filt2 = info_final['날짜'] == date

    df = info_final[filt1 & filt2]
    price = df['종목 자산액'].sum()
    prev_price=0

    prev_filt2 = info_final['날짜'] == prev_date
    prev_df = info_final[filt1 & prev_filt2]
    
    if prev_df.empty:
        for index,row in df.iterrows():
            k = row['로그 수익률']
            t1 = row['종목 자산액']
    
            t = t1 / math.exp(k)
            prev_price += t
    else:
        prev_price = prev_df['종목 자산액'].sum()
    
    earning = price / prev_price - 1

    new_row = {'etf코드' : ticker, '날짜' : date, '수익률' : earning}
    etf_earning.loc[len(etf_earning)] = new_row
    

In [1544]:
filt = etf_price['표준코드'] == 'KR7069660009'
etf_price[filt]

,단축코드,시가총액,날짜,표준코드
216,069660,688853000000,20190329,KR7069660009
596,069660,495827500000,20200331,KR7069660009
1011,069660,603630000000,20210331,KR7069660009
1506,069660,466107000000,20220331,KR7069660009
2117,069660,434136250000,20230331,KR7069660009
2881,069660,354748500000,20240329,KR7069660009


In [1550]:
etf_earning

,etf코드,날짜,수익률
0,KR7069500007,20240329,0.285791
1,KR7069660009,20240329,-0.182700
2,KR7091160002,20240329,1.821579
3,KR7091170001,20240329,0.333033
4,KR7091180000,20240329,0.010257
...,...,...,...
1269,KR7333980001,20200331,-0.139820
1270,KR7337120000,20200331,-0.168515
1271,KR7337140008,20200331,-0.137616
1272,KR7337150007,20200331,-0.225855


In [1642]:
etf_earning['1년간 코스피 수익률'] = None
etf_earning['1년간 코스피 std'] = None

#### 코스피 지수 가지고 오기

In [1643]:
import yfinance as yf

In [1644]:
kospi= yf.download(ticker,
                    start = "2019-03-28",
                    end = "2024-04-02")


[*********************100%%**********************]  1 of 1 completed


In [1645]:
kos = kospi['Close'].pct_change().dropna()

In [1646]:
date_list = pd.to_datetime(["2024-03-29" , "2023-03-31", "2022-03-31", "2021-03-31", "2020-03-31"])

In [1647]:
kos
kos_std = pd.DataFrame(columns = ['날짜','std'])

In [1648]:
last_index = kos.index[0]

for index, value in kos.items():
    if index in date_list:
        std = np.std(kos[last_index:index])
        position = kos.index.get_loc(index)

        new_row = {'날짜' : index, 'std' : std}
        kos_std.loc[len(kos_std)] = new_row
        
        last_index = kos.index[position+1]

In [1649]:
kos_std# 이건 코스피 지수 1년 동안 표준편차

,날짜,std
0,2020-03-31,0.015094
1,2021-03-31,0.013612
2,2022-03-31,0.009323
3,2023-03-31,0.011262
4,2024-03-29,0.009729


In [1659]:
new_date = pd.to_datetime(["2019-03-29"])
date_list = date_list.append(new_date)

In [1660]:
prices = kospi[kospi.index.isin(date_list)]['Close']

In [1661]:
prices

Date
2019-03-29    2140.669922
2020-03-31    1754.640015
2021-03-31    3061.419922
2022-03-31    2757.649902
2023-03-31    2476.860107
2024-03-29    2746.629883
Name: Close, dtype: float64

In [1662]:
ret = prices.pct_change().dropna()

In [1663]:
ret # 이건 코스프 지수 1년간의 상승률

Date
2020-03-31   -0.180331
2021-03-31    0.744757
2022-03-31   -0.099225
2023-03-31   -0.101822
2024-03-29    0.108916
Name: Close, dtype: float64

# 베타 값 구하기

In [1664]:
date_dic = {
    20240329 : '2024-03-29',
    20230331 : '2023-03-31',
    20220331 : '2022-03-31',
    20210331 : '2021-03-31',
    20200331 : '2020-03-31'
}

In [1667]:
for index, row in etf_earning.iterrows():
    date = row['날짜']
    kospi_earning = ret[date_dic[date]]
    kospi_std = kos_std[kos_std['날짜'] == date_dic[date]]['std'].values[0]
    etf_earning.at[index,'1년간 코스피 수익률'] = kospi_earning
    etf_earning.at[index,'1년간 코스피 std'] = kospi_std

In [1668]:
etf_earning

,etf코드,날짜,수익률,1년간 코스피 수익률,1년간 코스피 std
0,KR7069500007,20240329,0.285791,0.108916,0.009729
1,KR7069660009,20240329,-0.182700,0.108916,0.009729
2,KR7091160002,20240329,1.821579,0.108916,0.009729
3,KR7091170001,20240329,0.333033,0.108916,0.009729
4,KR7091180000,20240329,0.010257,0.108916,0.009729
...,...,...,...,...,...
1269,KR7333980001,20200331,-0.139820,-0.180331,0.015094
1270,KR7337120000,20200331,-0.168515,-0.180331,0.015094
1271,KR7337140008,20200331,-0.137616,-0.180331,0.015094
1272,KR7337150007,20200331,-0.225855,-0.180331,0.015094


In [1671]:
etf_earning.to_csv('etf 수익률 및 1년간 코스피 관련 지표.csv',index = False)

In [1674]:
info_final

,구성비중,etf코드,종목코드,날짜,시장구분,업종명,시가총액,주식 종류,PER,PBR,PCR,PSR,종가,종목 자산액,로그 수익률
0,31.12,KR7069500007,005930,20240329,KOSPI,전기전자,4.919101e+14,보통주,36.84169,1.50955,12.08101,2.05929,82400.0,2300595792000.0,0.252702
1,8.22,KR7069500007,000660,20240329,KOSPI,전기전자,1.332244e+14,보통주,-11.3046,1.81988,24.07848,3.14391,183000.0,607676652000.000122,0.725354
2,2.61,KR7069500007,005380,20240329,KOSPI,운수장비,4.928684e+13,보통주,4.6686,0.57835,-22.17142,0.34331,233000.0,192948426000.0,0.233389
3,2.57,KR7069500007,068270,20240329,KOSPI,의약품,4.169111e+13,보통주,55.3685,2.50701,55.21279,13.62691,191200.0,189991361999.999969,0.244019
4,2.29,KR7069500007,005490,20240329,KOSPI,철강금속,3.568906e+13,보통주,24.87694,0.69951,6.84913,0.54771,422000.0,169291914000.0,0.136922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121797,0.04,KR7337160006,002270,20200331,KOSPI,음식료품,3.395610e+11,보통주,12.46,0.55,5.26,0.26,300000.0,2641600.0,-0.749843
121798,0.03,KR7337160006,071840,20200331,KOSPI,유통업,3.694607e+11,보통주,-7.30155,0.37956,5.59604,0.18117,15650.0,1981200.0,-1.185269
121799,0.02,KR7337160006,019680,20200331,KOSPI,서비스업,5.005938e+11,보통주,37.64045,0.86124,8.45308,0.83365,5910.0,1320800.0,-0.046289
121800,0.02,KR7337160006,079430,20200331,KOSPI,기타제조업,1.458005e+11,보통주,14.08357,0.58636,2.86629,0.22152,7100.0,1320800.0,-1.062766


In [1675]:
etf_earning

,etf코드,날짜,수익률,1년간 코스피 수익률,1년간 코스피 std
0,KR7069500007,20240329,0.285791,0.108916,0.009729
1,KR7069660009,20240329,-0.182700,0.108916,0.009729
2,KR7091160002,20240329,1.821579,0.108916,0.009729
3,KR7091170001,20240329,0.333033,0.108916,0.009729
4,KR7091180000,20240329,0.010257,0.108916,0.009729
...,...,...,...,...,...
1269,KR7333980001,20200331,-0.139820,-0.180331,0.015094
1270,KR7337120000,20200331,-0.168515,-0.180331,0.015094
1271,KR7337140008,20200331,-0.137616,-0.180331,0.015094
1272,KR7337150007,20200331,-0.225855,-0.180331,0.015094
